In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hotel_canncelation/first inten project.csv')

In [ ]:
data.head()

# **Check for Null values**

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
# Changing datatype to Datetime
data['date of reservation'] =  pd.to_datetime(data['date of reservation'], errors='coerce')

# Extract day, month, and year from the date
data['year'] = data['date of reservation'].dt.year
data['month'] = data['date of reservation'].dt.month
data['day'] = data['date of reservation'].dt.day


# Dropping Booking_ID Column and date of reservation Column
data.drop(columns=['Booking_ID','date of reservation'] , inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data['year'] = data['year'].astype(int)
data['month'] = data['month'].astype(int)
data['day'] = data['day'].astype(int)

In [ ]:
data.info()

In [ ]:
# shape befor outlairs
data.shape

In [ ]:
# befor removing outliers
plt.figure(figsize=(12, 6))
sns.boxplot(data, orient='h')
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.groupby(['year','booking status'])['year'].count()
# Drop 2015,2016

In [ ]:
data = data[data['year'] != 2015]
data = data[data['year'] != 2016]

In [ ]:
data['average price ']

In [ ]:
data["average price "] = data["average price "].round().astype(int)

In [ ]:
outliers_cols = ["lead time", "average price "]
for column in outliers_cols:
    if data[column].dtype in ["int64", "float64"]:
        q1 = data[column].quantile(0.25)
        q3 = data[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        data = data[
            (data[column] >= lower_bound) & (data[column] <= upper_bound)
        ]

In [ ]:
# after removing outliers
plt.figure(figsize=(12, 6))
sns.boxplot(data, orient='h')
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.shape

In [ ]:
status_map = {'Canceled': 1, 'Not_Canceled': 0}
data['booking status'] = data['booking status'].map(status_map)

In [ ]:
# Identify categorical columns
encoding_columns = data.select_dtypes(include=["object"]).columns

# Perform one-hot encoding using pandas.get_dummies
df_encoded = pd.get_dummies(data, columns=encoding_columns)

# Print information about the encoded DataFrame
df_encoded.info()

In [ ]:
# Select boolean columns
boolean_columns = df_encoded.select_dtypes(include=["bool"]).columns

# Convert boolean columns to integer type (True -> 1, False -> 0)
df_encoded[boolean_columns] = df_encoded[boolean_columns].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Identify features and target
X = df_encoded.drop(columns=['booking status'])  # Features
y = df_encoded['booking status']  # Target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 15, 19, 25],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)